# Verilog Parsar

## Introduction
- This is a verilog parser written in python.
- It can parse verilog file and generate a report

## Usage
- Check for the following:
    - Unreachable Blocks
    - Uninitialized Register
    - Inferring Latches
    - Unreachable State
    - Non Full Case
    - Non Parallel Case
    - Multiple Drivers
    - Arithmetic Overflow
    - Integer Overflow

In [9]:
def file_reader(file_name):
    x = []
    try:
        # Open the file in read mode
        with open(file_name, 'r') as file:
            # Read and print each line
            for line in file:
                x.append(line.strip())  

    except FileNotFoundError:
        print(f"File not found: {file_name}")

    except Exception as e:
        print(f"An error occurred: {e}")
    
    return x


### Testing reading verilog file

In [10]:
file_path = 'tst.v'

y = file_reader(file_path)
    
for i in y:
        print(i)

module UnreachableBlocks();
reg reach;
wire state;
wire data;

initial
begin
reach = 1'b1;
end

always @(state)
begin
if (reach == 2'b0)
begin
data <= 1'b1;
end
end
endmodule

module UninitializedRegister();
wire data;

initial
begin
$display("%b", data);
end
endmodule

module InferringLatches();
reg out;
wire enable, Data;

always @(enable)
begin
if (enable)
begin
out <= Data;
end
end
endmodule

module UnreachableState(input clk);
reg [1:0] current_state, next_state;
localparam [1:0] S1 = 2'b00 ;
localparam [1:0] S2 = 2'b01 ;
localparam [1:0] S3 = 2'b10 ;

always @(posedge clk)
begin
current_state <= next_state;
end

always @(*)
begin
case (current_state)
S1:
begin
next_state <= S2;
end
S2:
begin
next_state <= S1;
end
S3
begin
next_state <= S1;
end

endcase
end
endmodule

module NonFullCase();
reg [1:0] x, y;

always @(*)
begin
case(x)
2'b00: y = 1'b00;
2'b01: y = 1'b01;
// Missing cases for '10' & '11'
endcase
end
endmodule

module NonParallelCase();
reg [1:0] x, y;

always @(*)
begi

### Unreachable State

### Non-Full Case

### Unintialized registers

### Inferring Latches

### Unreachable Blocks

### Multiple Drivers

### Arithmetic Overflow

### Non-Parallel Case